!pip install git+https://github.com/mfiume/omics-ai-python-library.git --quiet
from omics_ai import list_collections, list_tables, get_schema_fields, query
print("✅ Ready!")

## Connect to HiFi Solves"

In [ ]:
# List collections on HiFi Solves
collections = list_collections("hifisolves")
print(f"✅ Connected! Found {len(collections)} collections")

## Explore CoLoRS collection

In [ ]:
# Explore the CoLoRS collection (Consortium of Long Read Sequencing)
collection_slug = "consortium-of-long-read-sequencing-colors"
print(f"📂 Exploring CoLoRS collection: {collection_slug}")

# List tables in this collection
tables = list_tables("hifisolves", collection_slug)
print(f"📊 Found {len(tables)} tables:")

for i, table in enumerate(tables):
    name = table.get('display_name', table.get('name', 'Unknown'))
    size = table.get('size', 'Unknown')
    print(f"  {i+1}. {name} - {size:,} rows" if isinstance(size, int) else f"  {i+1}. {name} - {size} rows")

## Query genomics data

In [ ]:
# Query the CoLoRS small variants table
table_name = "collections.consortium_of_long_read_sequencing_colors.colors_small_variants"
print(f"🔬 Querying: {table_name}")

# Get first 10 variants
result = query("hifisolves", collection_slug, table_name, limit=10)

data = result.get('data', [])
print(f"✅ Retrieved {len(data)} variant records")

if data:
    # Show sample variant
    variant = data[0]
    print(f"\n📋 Sample variant fields:")
    for key, value in list(variant.items())[:8]:  # Show first 8 fields
        print(f"  {key}: {value}")
    
    print(f"\n📊 Total fields per variant: {len(variant)}")
    
    # Show pagination info if available
    if 'pagination' in result:
        total = result['pagination'].get('total', 'Unknown')
        print(f"📈 Total variants in table: {total:,}" if isinstance(total, int) else f"📈 Total variants: {total}")
else:
    print("❌ No data returned")

## Inspect schema

In [ ]:
# Explore the table schema
print("🔍 Table Schema:")
schema_fields = get_schema_fields("hifisolves", collection_slug, table_name)

print(f"📊 {len(schema_fields)} fields total")
print(f"\n📋 Key genomics fields:")

# Show important genomics fields
key_fields = ['chrom', 'pos', 'ref', 'alt', 'gene', 'consequence', 'sample_id', 'qual', 'filter']
for field in schema_fields:
    if any(key in field['field'].lower() for key in key_fields):
        print(f"  {field['field']}: {field['type']}")

print(f"\n💡 Full schema: {len(schema_fields)} fields available for filtering and analysis")

## Other networks

In [ ]:
# Try other networks
networks = {
    "Neuroscience AI": "neuroscience",
    "Viral AI": "viral", 
    "Biomedical AI": "biomedical",
    "ASAP (Parkinson's)": "asap"
}

print("🌐 Other Omics AI Explorer networks:")
for name, network in networks.items():
    try:
        collections = list_collections(network)
        print(f"  ✅ {name}: {len(collections)} collections")
    except Exception as e:
        print(f"  ❌ {name}: {str(e)[:50]}...")

## Authentication (optional)

In [ ]:
# If you have an access token, pass it to any function:
# collections = list_collections("hifisolves", access_token="your-token")
# result = query("hifisolves", collection_slug, table_name, access_token="your-token")

print("💡 See README for authentication details")

---\n\n**That's it!** 🎉\n\n- **Collection**: [CoLoRS (Consortium of Long Read Sequencing)](https://hifisolves.org/collections/consortium-of-long-read-sequencing-colors)\n- **Data**: Small variants from long-read sequencing studies\n- **GitHub**: [mfiume/omics-ai-python-library](https://github.com/mfiume/omics-ai-python-library)\n- **Networks**: hifisolves.org, neuroscience.ai, viral.ai, biomedical.ai"